In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
data20 = fetch_20newsgroups(subset='all', shuffle=True, remove=('headers', 'footers', 'quotes'))

In [3]:
list(enumerate(data20.target_names))

[(0, 'alt.atheism'),
 (1, 'comp.graphics'),
 (2, 'comp.os.ms-windows.misc'),
 (3, 'comp.sys.ibm.pc.hardware'),
 (4, 'comp.sys.mac.hardware'),
 (5, 'comp.windows.x'),
 (6, 'misc.forsale'),
 (7, 'rec.autos'),
 (8, 'rec.motorcycles'),
 (9, 'rec.sport.baseball'),
 (10, 'rec.sport.hockey'),
 (11, 'sci.crypt'),
 (12, 'sci.electronics'),
 (13, 'sci.med'),
 (14, 'sci.space'),
 (15, 'soc.religion.christian'),
 (16, 'talk.politics.guns'),
 (17, 'talk.politics.mideast'),
 (18, 'talk.politics.misc'),
 (19, 'talk.religion.misc')]

In [ ]:
import numpy as np
baseball = np.where(data20.target == 9)[0]
hockey = np.where(data20.target == 10)[0]

In [4]:
import numpy as np
pc = np.where(data20.target == 3)[0]
mac = np.where(data20.target == 4)[0]

In [ ]:
len(baseball), len(hockey)

In [ ]:
import numpy as np
ath = np.where(data20.target == 0)[0]
chrs = np.where(data20.target == 15)[0]

In [ ]:
strings = [data20.data[i] for i in list(ath) + list(chrs)]
target = [0 if data20.target[i] == 0 else 1 for i in list(ath) + list(chrs)]

In [ ]:
from vectorizer import cleaner
import re

def cleaner_20(text) :
    text = cleaner(text)
    text = re.sub(r'(\W)+', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

strings = [cleaner_20(s) for s in strings]

In [ ]:
import vectorizer
vec = vectorizer.Vectorizer(min_df=2)

In [ ]:
vec.fit(strings)

In [ ]:
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(range(len(strings)), stratify=target, test_size=0.2, random_state=13478)

In [ ]:
X_train = [strings[i] for i in train_idx]
X_test = [strings[i] for i in test_idx]

y_train = [target[i] for i in train_idx]
y_test = [target[i] for i in test_idx]

In [ ]:
vec.seq_text = {}
vec.seq_text['train'] = vec.get_seq_for_docs(X_train)
vec.seq_text['test'] = vec.get_seq_for_docs(X_test)

In [ ]:
vec.label = { 'train' : y_train, 'test' : y_test } 

In [ ]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
vectors = Vectors('wiki.simple.vec', url=url)

In [ ]:
vec.word_dim = vectors.dim

In [ ]:
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [ ]:
import pickle
pickle.dump(vec, open('20News/vec_relg.p', 'wb'))